In [1]:
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
import sys

In [2]:
with open("/Volumes/Drive/GitHub/DaVinciCodeTheTrackOfRobertLangdon/models/roberta-results", "rb") as fp:   # Unpickling
    bert_tags = pickle.load(fp)

In [3]:
bert_tags

[(' The Da Vinci Code', 'MISC', 1),
 (' Dan Brown', 'PER', 1),
 (' Louvre Museum', 'LOC', 1),
 (' Paris', 'LOC', 1),
 (' Jacques Sauniere', 'PER', 1),
 (' Grand Gallery', 'LOC', 1),
 (' Caravaggio', 'PER', 2),
 (' Sauniere', 'PER', 3),
 (' albino', 'MISC', 16),
 (' Sauniere', 'PER', 31),
 (' Sauniere', 'PER', 33),
 (' Sauniere', 'PER', 44),
 (' Sauniere', 'PER', 51),
 (' senechaux', 'PER', 51),
 (' Sauniere', 'PER', 64),
 (' Sauniere', 'PER', 65),
 (' Sauniere', 'PER', 66),
 (' Sauniere', 'PER', 70),
 (" Guerre d'Algerie", 'MISC', 76),
 ('.', 'MISC', 76),
 (' Jacques Sauniere', 'PER', 80),
 (' Jacques Sauniere', 'PER', 97),
 (' Grand Gallery', 'LOC', 103),
 (' Sauniere', 'PER', 104),
 (' Robert Langdon', 'PER', 108),
 (' Renaissance', 'MISC', 111),
 (' Louis XVI', 'MISC', 111),
 (' HOTEL RITZ PARIS', 'LOC', 113),
 (' Langdon', 'PER', 114),
 (' Langdon', 'PER', 116),
 (' Langdon', 'PER', 119),
 (' Langdon', 'PER', 124),
 (' AMERICAN UNIVERSITY OF PARIS', 'ORG', 127),
 (' ROBERT LANGDON'

In [15]:
def find_closest_loc(tag, index):
    # finds closest location to index
    curr_loc = ''
    dist = sys.maxsize
    for i in range(len(tag)):
        if bert_tags[i][1] == "LOC":
            if abs(bert_tags[i][2]-index) < dist:
                dist = abs(bert_tags[i][2]-index)
                curr_loc = bert_tags[i][0]
    return curr_loc

In [43]:
# go through bert_tags and map person to location based on closest distance index
pers_loc_temp = []
for i in tqdm(range(len(bert_tags))):
    if bert_tags[i][1] == "PER":
        loc = find_closest_loc(bert_tags, bert_tags[i][2])
        pers_loc_temp.append([bert_tags[i][0], loc])

pers_loc = []
# remove duplicates in pers_loc if index follows directly
for i in range(len(pers_loc_temp)-1):
    if not(pers_loc_temp[i][0] == pers_loc_temp[i+1][0] and pers_loc_temp[i][1] == pers_loc_temp[i+1][1]):
        pers_loc.append(pers_loc_temp[i])
pers_loc

100%|██████████| 116/116 [00:00<00:00, 187130.49it/s]


[[' Dan Brown', ' Louvre Museum'],
 [' Jacques Sauniere', ' Louvre Museum'],
 [' Caravaggio', ' Louvre Museum'],
 [' Sauniere', ' Louvre Museum'],
 [' senechaux', ' Louvre Museum'],
 [' Sauniere', ' Grand Gallery'],
 [' Jacques Sauniere', ' Grand Gallery'],
 [' Sauniere', ' Grand Gallery'],
 [' Robert Langdon', ' Grand Gallery'],
 [' Langdon', ' HOTEL RITZ PARIS'],
 [' Langdon', ' Chartres Cathedral'],
 [' ROBERT LANGDON', ' Chartres Cathedral'],
 [' Langdon', ' Chartres Cathedral'],
 [' Langdon', ' Vatican'],
 [' Langdon', ' Paris'],
 [' Langdon', ' CITY OF LIGHTS'],
 [' Robert', ' CITY OF LIGHTS'],
 [' Langdon', ' CITY OF LIGHTS'],
 [' Langdon', ' Pavilion Dauphine'],
 [' Langdon', ' Vatican'],
 ['Harrison Ford', ' Vatican'],
 [' Monique', ' Vatican'],
 [' Langdon', ' Vatican'],
 [' Langdon', ' Louvre'],
 [' Jerome Collet', ' Louvre'],
 [' Langdon', ' Louvre'],
 [' Jacques Sauniere', ' Louvre'],
 [' Langdon', ' Louvre'],
 [' Sauniere', ' Louvre'],
 [' Langdon', ' Louvre'],
 [' Langdo

In [44]:
vectorizer = TfidfVectorizer()


def compute_similarity(a, b):
  tfidf = vectorizer.fit_transform([a, b])
  similarity = ((tfidf * tfidf.T).toarray())[0,1]
  return similarity

In [45]:
characters = ["Jacques Saunière", "Manuel Aringarosa", "Sister Sandrine Bieil", "Marie Chauvel", "Jerome Collet", "Simon Edwards", "Bezu Fache", "Jonas Faukman", "Pamela Gettum", "Claude Grouard", "Robert Langdon", "Rémy Legaludec", "Sophie Neveu"]

In [46]:
unique_chars = set()
threshold = 0.3

# Compute the similarity between the characters and the persons
for i in range(len(pers_loc)):
  for j in range(len(characters)):
    temp = []
    char1 = pers_loc[i][0].lower()
    char2 = characters[j].lower()
    distance = compute_similarity(char1, char2)
    if distance > threshold:
      unique_chars.add(char2)
      pers_loc[i][0] = char2
      break
unique_chars

{'jacques saunière', 'jerome collet', 'robert langdon'}

In [47]:
pers_loc

[[' Dan Brown', ' Louvre Museum'],
 ['jacques saunière', ' Louvre Museum'],
 [' Caravaggio', ' Louvre Museum'],
 [' Sauniere', ' Louvre Museum'],
 [' senechaux', ' Louvre Museum'],
 [' Sauniere', ' Grand Gallery'],
 ['jacques saunière', ' Grand Gallery'],
 [' Sauniere', ' Grand Gallery'],
 ['robert langdon', ' Grand Gallery'],
 ['robert langdon', ' HOTEL RITZ PARIS'],
 ['robert langdon', ' Chartres Cathedral'],
 ['robert langdon', ' Chartres Cathedral'],
 ['robert langdon', ' Chartres Cathedral'],
 ['robert langdon', ' Vatican'],
 ['robert langdon', ' Paris'],
 ['robert langdon', ' CITY OF LIGHTS'],
 ['robert langdon', ' CITY OF LIGHTS'],
 ['robert langdon', ' CITY OF LIGHTS'],
 ['robert langdon', ' Pavilion Dauphine'],
 ['robert langdon', ' Vatican'],
 ['Harrison Ford', ' Vatican'],
 [' Monique', ' Vatican'],
 ['robert langdon', ' Vatican'],
 ['robert langdon', ' Louvre'],
 ['jerome collet', ' Louvre'],
 ['robert langdon', ' Louvre'],
 ['jacques saunière', ' Louvre'],
 ['robert langdo